### Docs
* https://developers.google.com/youtube/v3/docs

In [ ]:
# -*- coding: utf-8 -*-
import os, json, flask, requests, pandas as pd

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
import google.auth.transport.requests  
from googleapiclient.http import MediaFileUpload 

In [ ]:
def run_me_to_refresh_the_access_token(): 
    with open('oauth2_user_credentials.json') as fr:
        credentials = google.oauth2.credentials.Credentials(**json.load(fp=fr)) 
        
    # The old access token is still vaild even you got a new valid access token. 
        
    with open('oauth2_user_credentials.json', 'w') as fw:
        credentials.refresh(google.auth.transport.requests.Request())
        creden_js = credentials.to_json()
        refreshed_credentials = json.loads(credentials.to_json())
        # we shall discard expiry before save. Try and see the consequence if you shall not.
        #refreshed_credentials.pop('expiry') 
        json.dump(refreshed_credentials, fp=fw)

In [ ]:
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
with open('oauth2_user_credentials.json') as fr:
    credentials = google.oauth2.credentials.Credentials(**json.load(fp=fr))
    
youtube = googleapiclient.discovery.build(API_SERVICE_NAME, 
                                          API_VERSION, 
                                          credentials=credentials)

In [ ]:
# I am going to upload my video to this playlist "PLUenxxxx_xxxxfrNm92hxxxxQdvukmViZ"
playlistId = open('playerlist_id.txt').read() 
myUploadedVideo, response = {}, {}
while True:
    response = youtube.playlistItems().list(
        playlistId=playlistId,
        part='snippet',
        maxResults=50,
        pageToken=response.get('nextPageToken', '')
    ).execute()  
    # Print information about each video. 
    for playlist_item in response['items']:
        title = playlist_item['snippet']['title']
        video_id = playlist_item['snippet']['resourceId']['videoId']
        myUploadedVideo[title]=video_id   
    if not response.get('nextPageToken', ''): 
        break # no next page then break 

In [ ]:
df = pd.read_csv('audio/list.csv')
df['title'] = df.apply(lambda x: '梁文道 八分 '+x.title, axis=1) 
df = df[~df['title'].isin(myUploadedVideo)] 
df = df.set_index('vid') 
df = df[df['t_end'].notna()] 
print(df.index)
display(df)

In [ ]:
myUploadedVideo

In [ ]:
for vid, row in df.iterrows(): 
    audio_file=f'video/{vid}.mp4' 
    if not os.path.exists(audio_file):
        print('[Video File Does not Exist]', audio_file)
        continue
    title = row['title']
    print(f'Uploading {title} ... ...') 
    body={
        'snippet':{
            'title':title,
            #'description':'More words for this video',
            'tags':['梁文道','八分'],
            'categoryId':'22' # https://developers.google.com/youtube/v3/docs/videoCategories/list
        },
        'status':{
            'privacyStatus':'public', # ['private','public','unlisted']
            'selfDeclaredMadeForKids':False
        }
    }
    uploadedVideo = youtube.videos().insert(
        part=','.join(body.keys()),
        body=body,
        media_body=MediaFileUpload(audio_file, chunksize=-1, resumable=False)
    ).execute()
    
    videoId   = uploadedVideo['id'] 
    
    body = {'snippet':{
        "playlistId": playlistId, # '梁文道－八分'
        #"position": 0,
        "resourceId": {
          "kind": "youtube#video",
          "videoId": videoId
        }
    }}
    
    resource = youtube.playlistItems().insert(part='snippet', body=body).execute()
    print(f'{title} Uploaded!')